In [2]:
import re
import numpy as np
import gensim.downloader as api
from gensim.utils import simple_preprocess
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS

word2vec_model = api.load("word2vec-google-news-300")

import pandas as pd
file_path = 'Combined Data.csv'
df = pd.read_csv(file_path)
def preprocess_and_tokenize(text):
    if not isinstance(text, str):
        return []

    text = re.sub(r"[^A-Za-z0-9\s]", "", text.lower())
    tokens = simple_preprocess(text, deacc=True)
    tokens = [word for word in tokens if word not in ENGLISH_STOP_WORDS]
    return tokens

df['tokens'] = df['statement'].apply(preprocess_and_tokenize)
def get_average_word2vec(tokens, model, vector_size=300):
    vectors = [model[word] for word in tokens if word in model]
    return np.mean(vectors, axis=0) if vectors else np.zeros(vector_size)

df['word2vec_vector'] = df['tokens'].apply(lambda x: get_average_word2vec(x, word2vec_model))


df['status'] = df['status'].apply(lambda x: 0 if x == 'Normal' else 1)

df['status'].value_counts()

,count
status,
1,36692
0,16351


**HYBRID MODEL**

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
import torch
import numpy as np

X = np.stack(df['word2vec_vector'].values)
y = df['status'].values

X_tensor = torch.tensor(X, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.long)
X_train, X_test, y_train, y_test = train_test_split(X_tensor, y_tensor, test_size=0.2, random_state=42)

train_data = TensorDataset(X_train, y_train)
test_data = TensorDataset(X_test, y_test)

train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)



**TRANSFORMER+DT**

In [13]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.d_model = d_model
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-torch.log(torch.tensor(10000.0)) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        self.pe = pe.unsqueeze(0)

    def forward(self, x):
        return x + self.pe[:, :x.size(1), :].to(x.device)

class TransformerFeatureExtractor(nn.Module):
    def __init__(self, input_size, d_model=128, num_heads=4, num_layers=2):
        super(TransformerFeatureExtractor, self).__init__()
        self.embedding = nn.Linear(input_size, d_model)
        self.pos_encoder = PositionalEncoding(d_model)
        encoder_layers = nn.TransformerEncoderLayer(d_model=d_model, nhead=num_heads, dim_feedforward=256, dropout=0.1, batch_first=True)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layers, num_layers=num_layers)

    def forward(self, x):
        x = self.embedding(x).unsqueeze(1)
        x = self.pos_encoder(x)
        x = self.transformer_encoder(x)
        return x.mean(dim=1)

class SoftDecisionTree(nn.Module):
    def __init__(self, input_dim, num_classes, num_nodes=10):
        super(SoftDecisionTree, self).__init__()
        self.inner_nodes = nn.Linear(input_dim, num_nodes)
        self.leaf_nodes = nn.Linear(num_nodes, num_classes)

    def forward(self, x):
        decision_weights = torch.sigmoid(self.inner_nodes(x))
        leaf_outputs = self.leaf_nodes(decision_weights)
        return leaf_outputs

class TransformerDecisionTree(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_classes, num_nodes=10):
        super(TransformerDecisionTree, self).__init__()
        self.feature_extractor = TransformerFeatureExtractor(input_dim, hidden_dim)
        self.decision_tree = SoftDecisionTree(hidden_dim, num_classes, num_nodes)

    def forward(self, x):
        features = self.feature_extractor(x)
        output = self.decision_tree(features)
        return output

class CustomDataset(data.Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X).clone().detach().float()
        self.y = torch.tensor(y).clone().detach().long()

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

def train_model(model, train_loader, test_loader, epochs=10, lr=0.001):
    model.to(device)
    optimizer = optim.Adam(model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()

    for epoch in range(epochs):
        model.train()
        total_loss = 0
        correct = 0
        total = 0

        for X_batch, y_batch in train_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            optimizer.zero_grad()
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += y_batch.size(0)
            correct += (predicted == y_batch).sum().item()

        train_acc = 100 * correct / total
        print(f"Epoch {epoch+1}, Loss: {total_loss/len(train_loader):.4f}, Train Accuracy: {train_acc:.2f}%")

    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for X_batch, y_batch in test_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            outputs = model(X_batch)
            _, predicted = torch.max(outputs, 1)
            total += y_batch.size(0)
            correct += (predicted == y_batch).sum().item()

    print(f"Test Accuracy: {100 * correct / total:.2f}%")

batch_size = 64
train_dataset = CustomDataset(X_train, y_train)
test_dataset = CustomDataset(X_test, y_test)
train_loader = data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

input_dim = X_train.shape[1]
num_classes = len(torch.unique(y_train))
model = TransformerDecisionTree(input_dim=input_dim, hidden_dim=128, num_classes=num_classes, num_nodes=10)

train_model(model, train_loader, test_loader, epochs=10, lr=0.001)


<ipython-input-13-ae4d7bff8434>:61: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.X = torch.tensor(X).clone().detach().float()
<ipython-input-13-ae4d7bff8434>:62: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.y = torch.tensor(y).clone().detach().long()  # Ensure integer labels for CrossEntropyLoss


Epoch 1, Loss: 0.3660, Train Accuracy: 85.98%
Epoch 2, Loss: 0.3215, Train Accuracy: 87.71%
Epoch 3, Loss: 0.3119, Train Accuracy: 88.12%
Epoch 4, Loss: 0.2883, Train Accuracy: 88.98%
Epoch 5, Loss: 0.2759, Train Accuracy: 89.40%
Epoch 6, Loss: 0.2701, Train Accuracy: 89.64%
Epoch 7, Loss: 0.2582, Train Accuracy: 89.87%
Epoch 8, Loss: 0.2511, Train Accuracy: 90.20%
Epoch 9, Loss: 0.2436, Train Accuracy: 90.39%
Epoch 10, Loss: 0.2350, Train Accuracy: 90.77%
Test Accuracy: 90.38%


**TRANSFOMER + SVM**

In [14]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.d_model = d_model
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-torch.log(torch.tensor(10000.0)) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        self.pe = pe.unsqueeze(0)

    def forward(self, x):
        return x + self.pe[:, :x.size(1), :].to(x.device)

class TransformerFeatureExtractor(nn.Module):
    def __init__(self, input_size, d_model=128, num_heads=4, num_layers=2):
        super(TransformerFeatureExtractor, self).__init__()
        self.embedding = nn.Linear(input_size, d_model)
        self.pos_encoder = PositionalEncoding(d_model)
        encoder_layers = nn.TransformerEncoderLayer(d_model=d_model, nhead=num_heads, dim_feedforward=256, dropout=0.1, batch_first=True)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layers, num_layers=num_layers)

    def forward(self, x):
        x = self.embedding(x).unsqueeze(1)
        x = self.pos_encoder(x)
        x = self.transformer_encoder(x)
        return x.mean(dim=1)

class SoftSVM(nn.Module):
    def __init__(self, input_dim, num_classes):
        super(SoftSVM, self).__init__()
        self.fc = nn.Linear(input_dim, num_classes)

    def forward(self, x):
        return self.fc(x)

class TransformerSVM(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_classes):
        super(TransformerSVM, self).__init__()
        self.feature_extractor = TransformerFeatureExtractor(input_dim, hidden_dim)
        self.svm = SoftSVM(hidden_dim, num_classes)

    def forward(self, x):
        features = self.feature_extractor(x)
        output = self.svm(features)
        return output

class CustomDataset(data.Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X).clone().detach().float()
        self.y = torch.tensor(y).clone().detach().long()

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

def train_model(model, train_loader, test_loader, epochs=10, lr=0.001):
    model.to(device)
    optimizer = optim.Adam(model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()

    for epoch in range(epochs):
        model.train()
        total_loss = 0
        correct = 0
        total = 0

        for X_batch, y_batch in train_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            optimizer.zero_grad()
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += y_batch.size(0)
            correct += (predicted == y_batch).sum().item()

        train_acc = 100 * correct / total
        print(f"Epoch {epoch+1}, Loss: {total_loss/len(train_loader):.4f}, Train Accuracy: {train_acc:.2f}%")

    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for X_batch, y_batch in test_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            outputs = model(X_batch)
            _, predicted = torch.max(outputs, 1)
            total += y_batch.size(0)
            correct += (predicted == y_batch).sum().item()

    print(f"Test Accuracy: {100 * correct / total:.2f}%")

batch_size = 64
train_dataset = CustomDataset(X_train, y_train)
test_dataset = CustomDataset(X_test, y_test)
train_loader = data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

input_dim = X_train.shape[1]
num_classes = len(torch.unique(y_train))
model = TransformerSVM(input_dim=input_dim, hidden_dim=128, num_classes=num_classes)

train_model(model, train_loader, test_loader, epochs=10, lr=0.001)


<ipython-input-14-f22a9e415b8b>:58: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.X = torch.tensor(X).clone().detach().float()
<ipython-input-14-f22a9e415b8b>:59: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.y = torch.tensor(y).clone().detach().long()


Epoch 1, Loss: 0.3505, Train Accuracy: 86.18%
Epoch 2, Loss: 0.2835, Train Accuracy: 88.97%
Epoch 3, Loss: 0.2509, Train Accuracy: 90.29%
Epoch 4, Loss: 0.2400, Train Accuracy: 90.70%
Epoch 5, Loss: 0.2281, Train Accuracy: 91.08%
Epoch 6, Loss: 0.2147, Train Accuracy: 91.67%
Epoch 7, Loss: 0.2065, Train Accuracy: 91.87%
Epoch 8, Loss: 0.2054, Train Accuracy: 92.06%
Epoch 9, Loss: 0.1994, Train Accuracy: 92.32%
Epoch 10, Loss: 0.1947, Train Accuracy: 92.54%
Test Accuracy: 90.25%


In [15]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.d_model = d_model
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-torch.log(torch.tensor(10000.0)) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        self.pe = pe.unsqueeze(0)

    def forward(self, x):
        return x + self.pe[:, :x.size(1), :].to(x.device)

class TransformerFeatureExtractor(nn.Module):
    def __init__(self, input_size, d_model=128, num_heads=4, num_layers=2):
        super(TransformerFeatureExtractor, self).__init__()
        self.embedding = nn.Linear(input_size, d_model)
        self.pos_encoder = PositionalEncoding(d_model)
        encoder_layers = nn.TransformerEncoderLayer(d_model=d_model, nhead=num_heads, dim_feedforward=256, dropout=0.1, batch_first=True)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layers, num_layers=num_layers)

    def forward(self, x):
        x = self.embedding(x).unsqueeze(1)
        x = self.pos_encoder(x)
        x = self.transformer_encoder(x)
        return x.mean(dim=1)

class SoftDecisionTree(nn.Module):
    def __init__(self, input_dim, num_classes, num_nodes=10):
        super(SoftDecisionTree, self).__init__()
        self.inner_nodes = nn.Linear(input_dim, num_nodes)
        self.leaf_nodes = nn.Linear(num_nodes, num_classes)

    def forward(self, x):
        decision_weights = torch.sigmoid(self.inner_nodes(x))
        leaf_outputs = self.leaf_nodes(decision_weights)
        return leaf_outputs

class SoftRandomForest(nn.Module):
    def __init__(self, input_dim, num_classes, num_trees=5, num_nodes=10):
        super(SoftRandomForest, self).__init__()
        self.trees = nn.ModuleList([SoftDecisionTree(input_dim, num_classes, num_nodes) for _ in range(num_trees)])

    def forward(self, x):
        tree_outputs = torch.stack([tree(x) for tree in self.trees], dim=0)
        return tree_outputs.mean(dim=0)

class TransformerRandomForest(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_classes, num_trees=5, num_nodes=10):
        super(TransformerRandomForest, self).__init__()
        self.feature_extractor = TransformerFeatureExtractor(input_dim, hidden_dim)
        self.random_forest = SoftRandomForest(hidden_dim, num_classes, num_trees, num_nodes)

    def forward(self, x):
        features = self.feature_extractor(x)
        output = self.random_forest(features)
        return output

class CustomDataset(data.Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X).clone().detach().float()
        self.y = torch.tensor(y).clone().detach().long()

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

def train_model(model, train_loader, test_loader, epochs=10, lr=0.001):
    model.to(device)
    optimizer = optim.Adam(model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()

    for epoch in range(epochs):
        model.train()
        total_loss = 0
        correct = 0
        total = 0

        for X_batch, y_batch in train_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            optimizer.zero_grad()
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += y_batch.size(0)
            correct += (predicted == y_batch).sum().item()

        train_acc = 100 * correct / total
        print(f"Epoch {epoch+1}, Loss: {total_loss/len(train_loader):.4f}, Train Accuracy: {train_acc:.2f}%")

    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for X_batch, y_batch in test_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            outputs = model(X_batch)
            _, predicted = torch.max(outputs, 1)
            total += y_batch.size(0)
            correct += (predicted == y_batch).sum().item()

    print(f"Test Accuracy: {100 * correct / total:.2f}%")

batch_size = 64
train_dataset = CustomDataset(X_train, y_train)
test_dataset = CustomDataset(X_test, y_test)
train_loader = data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

input_dim = X_train.shape[1]
num_classes = len(torch.unique(y_train))
model = TransformerRandomForest(input_dim=input_dim, hidden_dim=128, num_classes=num_classes, num_trees=5, num_nodes=10)

train_model(model, train_loader, test_loader, epochs=10, lr=0.001)


<ipython-input-15-0ca3a5882a93>:70: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.X = torch.tensor(X).clone().detach().float()
<ipython-input-15-0ca3a5882a93>:71: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.y = torch.tensor(y).clone().detach().long()


Epoch 1, Loss: 0.4099, Train Accuracy: 83.17%
Epoch 2, Loss: 0.3209, Train Accuracy: 87.85%
Epoch 3, Loss: 0.2895, Train Accuracy: 89.08%
Epoch 4, Loss: 0.2714, Train Accuracy: 89.62%
Epoch 5, Loss: 0.2613, Train Accuracy: 89.97%
Epoch 6, Loss: 0.2550, Train Accuracy: 90.09%
Epoch 7, Loss: 0.2506, Train Accuracy: 90.39%
Epoch 8, Loss: 0.2389, Train Accuracy: 90.68%
Epoch 9, Loss: 0.2322, Train Accuracy: 91.02%
Epoch 10, Loss: 0.2259, Train Accuracy: 91.32%
Test Accuracy: 90.51%
